In [31]:
# Setup
from huggingface_hub import hf_hub_download
from datasets import load_dataset
import time
import ray
import os
os.chdir("C:\\Users\\WulfN\\.cache\\huggingface\\hub\\")

%config InteractiveShell.ast_node_interactivity = 'all'
# download model
# hf_hub_download(repo_id = 'prithivMLmods/Deep-Fake-Detector-Model', filename = 'config.json') # presumably the config file is needed first
# model.safetensors, # pytorch_model.bin

# Datasets can be found here: https://huggingface.co/datasets?modality=modality:image&sort=trending&search=deepfake

### Load Data
dataset = load_dataset("Whab/deepfake")

# Enable multiprocessing 
ray.shutdown()
ray.init(num_cpus=os.cpu_count())
from ray.util.multiprocessing import Pool

2024-12-11 18:25:41,631	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.12.0
Ray version:,2.35.0


# Preview dataset images

In [ ]:
#import ipywidgets as widgets
# from IPython.display import display
# import ipyplot
#from PIL import Image
#import matplotlib.pyplot as plt

# preview_num = 5

# sample_images = dataset['train'][1:preview_num + 1]

# display_images = [
#     widgets.Image(value=sample_images['image'][i].tobytes(), format='png', width=256, height=256) 
#     for i in range(preview_num)
# ]

# display_images
# display(widgets.HBox(display_images))

## Display images vertically
# for i in range(preview_num):
#     display(sample_images['image'][i])

# display(display_images) 
# #widgets.HBox(display_images)
# # hbox = widgets.HBox(display_images)
# # display(hbox)
# #display_images

Function to create batches of image data

In [33]:
import pandas as pd
import numpy as np
# import modin as pd

def create_image_batches(data, batch_size):
    
    num_images = data.num_rows # data[0].num_rows may be needed when using starmap
    batch_size = batch_size

    # index value of first image in batch i
    batch_key = list(range(0, num_images, batch_size))

    # initializing result dataframe
    result = pd.DataFrame({'image_batch':[]})

    # batching images in groups of chunk_size
    for i in range(len(batch_key)):

        # index of all images in batch
        last_batch_adjustment = (num_images - max(batch_key))
        batch_indices = list(range(batch_key[i], batch_key[i] + last_batch_adjustment))

        # appending images from batch into list
        batched_images = []
        for j in batch_indices:
            batched_images.append(data[j]['image'])

        batch = pd.DataFrame({'image_batch': [batched_images]})
        
        # combining batches into pandas dataframe
        result = pd.concat([result, batch])
            
    return result


In [ ]:
# n_cores = os.cpu_count()
# test_df = dataset['train'].train_test_split(test_size=0.1, shuffle=False)

# chunk_6 = test_df['test'].shard(num_shards=n_cores, index=6)
# chunk_7 = test_df['test'].shard(num_shards=n_cores, index=7)

# data = chunk_7
# ########## DELETE ########## DELETE ########## DELETE ########## DELETE 
# num_images = data.num_rows # chunk_6, chunk_7
# batch_size = 3

# # index value of first image in batch i
# batch_key = list(range(0, num_images, batch_size))

# # initializing result dataframe
# result = pd.DataFrame({'index': [],
#                        'image_batch': []})

# # batching images in groups of chunk_size
# for i in range(len(batch_key)):

#     # index of all images in batch
#     last_batch_adjustment = (num_images - max(batch_key))
#     #if (max(batch_key) + batch_size - 1) > num_images
#     batch_indices = list(range(batch_key[i], batch_key[i] + last_batch_adjustment))
    

#     # appending images from batch into list
#     batched_images = []
#     for j in batch_indices:
#         batched_images.append(data[j]['image'])

#     batch = pd.DataFrame({'index': [i],
#                           'image_batch': [batched_images]})
    
#     # Adding batch to dataframe
#     result = pd.concat([result, batch])

# max(batch_key) # 2241
# last_batch_adjustment
# batch_indices
# chunk_7
# result


In [40]:
from ray.util.multiprocessing import Pool

n_cores = os.cpu_count()
test_df = dataset['train'].train_test_split(test_size=0.1, shuffle=False)

args = []
for i in range(n_cores):
    chunk = test_df['test'].shard(num_shards=n_cores, index=i)
    args.append((chunk, 3))

########################################### functionalize with the function and args as function arguments
if __name__ == "__main__":
    # Create arguments for starmap
    start = time.time()

    results = []
    pool = Pool(n_cores)
    # for result in pool.starmap(create_image_batches, args):
    #     results.append(result)
        #print(result) consider head of dataset?
    results = pool.starmap(create_image_batches, args)

    end = time.time() - start
    print('Process complete in: ', end)

results


Process complete in:  64.56059098243713


[                                          image_batch
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 ..                                                ...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 
 [748 rows x 1 columns],
                                           image_batch
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFile image mode...
 0   [<PIL.JpegImagePlugin.JpegImageFi

In [42]:
fn_only = create_image_batches(test_df['test'], batch_size = 3)

In [ ]:
# combine dataframes continuously
# functionalize ray starmap combo
# TEST parallelizing create_image_batches function when using model - 55-65 seconds on test_df vs 23 seconds without parallelization
# ensure you image batches can be fed to model
# test how long it takes with 1 image, batch of 2, and batch of 3, test batching all images at once beforehand, and including in the multiprocessing portion
# apply function to model training on the training data



0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
                           ...                        
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
0    [<PIL.JpegImagePlugin.JpegImageFile image mode...
Name: image_batch, Length: 748, dtype: object

In [ ]:
def simple_fn(x, y):
    time.sleep(2)
    return x + y

In [ ]:
# Goal: Map create_image_batches over each chunk with batch_size of 3
# with Pool(n_cores) as pool:
#     chunks = [test_df['test'].shard(num_shards=n_cores, index=i) for i in range(n_cores)] 
#     args = (chunks, 3)



pool = Pool(n_cores)#with Pool(n_cores) as pool:
chunks = [test_df['test'].shard(num_shards=n_cores, index=i) for i in range(n_cores)] 

# Create arguments for starmap
args = []
for i in range(n_cores):
    args.append((chunks[i], 3))


In [ ]:

# Rather than results_a = []pipe(image i), pipe(batch of images i), try batch of 3 images then 2
# predictions = pd.DataFrame()
# for i in range(batches):
#     image_score = pd.DataFrame(pipe(batches[i])) #dataset['train'][:df_length+1]['image'][i]
#     predictions = pd.concat([predictions, pd.concat({i: image_score}, names=['image_num'])])

In [ ]:
from transformers import pipeline
# Consider Fully Sharded Data Parallel on model for processing

batches_test = batches[:100]

# Huggingface model
pipe = pipeline("image-classification", model="prithivMLmods/Deep-Fake-Detector-Model", device=-1); 

# Get predictions over batches of images
predictions_test = parallel_apply(batches_test, 'image_batch', pipe)
predictions_test

In [ ]:
# Save table in repo
import pickle
predictions.to_pickle('huggingface_predictions')

dist'n of fake and real scores

In [ ]:
# dist'n of fake and real scores
import seaborn as sb
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Fake'], bins=100)
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Real'], bins=100)

In [ ]:
# AUC score